In [1]:
!pip install line_profiler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.8/714.8 kB 8.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [4]:
import numpy as np
# import line_profiler
%load_ext line_profiler
corpus = [ "The king ruled the kingdom with wisdom.", "The queen ruled the kingdom with grace.", "A wise man gave counsel to the king.", "A wise woman gave counsel to the queen.", "The king and the queen hosted a grand ball.", "The man went to the market.", "The woman went to the garden.", "The king and the man discussed politics.", "The queen and the woman discussed art.", "The king and the woman went for a walk.", "The man and the queen had a conversation.", "The wise king made important decisions.", "The wise woman offered valuable advice.", "The king and the man played chess.", "The queen and the woman painted together." ]


corpus = [
    'the quick brown fox jumped over the lazy dog',
    'the brown fox is quick and the blue dog is lazy',
    'the quick blue dog jumped over the lazy brown fox'
]
# Tokenize the sentences and build a vocabulary
tokens = [sentence.split() for sentence in corpus]
vocabulary = set(word for sentence in tokens for word in sentence)
word_to_index = {word: i for i, word in enumerate(vocabulary)}

# Create one-hot encoded vectors for each word in the vocabulary
vocab_size = len(vocabulary)
one_hot_encodings = np.eye(vocab_size)

# Define the context window size
window_size = 2
training_pairs = []

# Generate training pairs with one-hot encoding
for sentence in tokens:
    sentence_indices = [word_to_index[word] for word in sentence]
    for i, word_index in enumerate(sentence_indices):
        for j in range(max(i - window_size, 0), min(i + window_size + 1, len(sentence_indices))):
            if i != j:
                center_word_vector = one_hot_encodings[word_index]  # One-hot encoded center word vector
                context_word_vector = one_hot_encodings[sentence_indices[j]]  # One-hot encoded context word vector
                training_pairs.append((center_word_vector, context_word_vector))

# Define the Word2Vec model architecture (Skip-gram)
embedding_size = 5  # Small embedding size for simplicity
W = np.random.rand(vocab_size, embedding_size)  # Input to hidden layer weights (Vocab size x Embedding size)
W_prime = np.random.rand(embedding_size, vocab_size)  # Hidden to output layer weights (Embedding size x Vocab size)

# Training parameters
learning_rate = 0.01
epochs = 1

@profile
def training_code():
    # Training loop
    losses = []
    for epoch in range(epochs):
        loss = 0
        for center_word_vector, context_word_vector in training_pairs:
            # Forward pass
            h = np.dot(center_word_vector, W)  # Hidden layer output (1 x Embedding size)
            u = np.dot(h, W_prime)  # Score vector (1 x Vocab size)
            y_pred = np.exp(u) / sum(np.exp(u))  # Softmax output (1 x Vocab size)
            # Backward pass
            EI = y_pred - context_word_vector  # Error indicator vector (1 x Vocab size)
            dW_prime = np.outer(h, EI)  # Gradient for W_prime (Embedding size x Vocab size)
            dW = np.outer(center_word_vector, np.dot( W_prime, EI))  # Gradient for W (Vocab size x Embedding size)
            
            # Update weights
            W -= learning_rate * dW
            W_prime -= learning_rate * dW_prime
        losses.append(loss)


        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')


NameError: name 'profile' is not defined